In [ ]:
from opensky_api import OpenSkyApi, FlightTrack
import os.path
import pandas as pd
import folium
import time
import os 
from dotenv import load_dotenv
from folium.plugins import MarkerCluster
load_dotenv()
import avwx
opensky_user = os.getenv('OPENSKY_USER')
opensky_api_key = os.getenv('OPENSKY_API_KEY')

# Initialize the API (can also be authenticated) 
# dont be dumb if you have the keys
api = OpenSkyApi(opensky_user, opensky_api_key)

In [ ]:
# fetch flights only in the united states 
states = api.get_states()
data = []
for s in states.states:
    data.append({
            "ICAO24": s.icao24,
            "Callsign": s.callsign,
            "Origin Country": s.origin_country,
            "Latitude": s.latitude,
            "Longitude": s.longitude,
            "Baro Altitude": s.baro_altitude,
            "Geo Altitude": s.geo_altitude,
            "Velocity": s.velocity,
            "True Track": s.true_track,
            "Vertical Rate": s.vertical_rate,
            "On Ground": s.on_ground,
            "Category": s.category,
            "Last Contact": s.last_contact,
            "Position Source": s.position_source,
            "Sensors": s.sensors,
            "SPI": s.spi,
            "Squawk": s.squawk,
            "Time Position": s.time_position
        })
df = pd.DataFrame(data)
df

In [ ]:
# Assuming 'data' is populated as per your existing code snippet
us_flights_data = [flight for flight in data if flight["Origin Country"] == "United States"]
us_flights_df = pd.DataFrame(us_flights_data)
us_flights_df

In [ ]:

# Load the airport data from the CSV file
airport_df = pd.read_csv('airports.csv')
# US only airports
us_airports_df = airport_df[airport_df['iso_country'] == 'US']
# now only get medium airport and large airports
us_airports_df = us_airports_df[us_airports_df['type'].isin(['medium_airport', 'large_airport'])]
large_airports_df = us_airports_df[us_airports_df['type'] == 'large_airport']

In [ ]:
large_airports_df

# Weather: OpenWeatherMap API

In [ ]:
import requests

def fetch_weather_openweathermap(api_key, lat, lon):
    """
    Fetch current weather data for the specified latitude and longitude using OpenWeatherMap.
    """
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch weather data: {response.status_code}")
        return None

# Example usage
api_key = os.getenv('OPENWEATHERMAP_API_KEY')

# Assuming you have a DataFrame 'us_airports_df' with filtered US airports
for index, row in us_airports_df.iterrows():
    lat, lon = map(float, row['coordinates'].split(', '))
    weather_data = fetch_weather_openweathermap(api_key, lat, lon)
    print(weather_data)  # Process or store this data as needed


# Weather: AVWX library

In [25]:
import avwx
from avwx.forecast import *
import asyncio

# Function to Fetch Data
async def fetch_airport_data_async(airport_codes):
    airport_data = {}
    for airport_code in airport_codes:
        data = {}
        # Fetch METAR data
        metar = avwx.Metar(airport_code)
        if await metar.async_update():
            data["metar"] = {
                "raw": metar.raw,
                "flight_rules": metar.data.flight_rules,
                "summary": metar.summary, 
            }
        else:
            data["metar"] = None

        # Fetch TAF data
        taf = avwx.Taf(airport_code)
        if await taf.async_update():
            data["taf"] = {
                "raw": taf.translations,
                "forecast": [
                    {
                        "flight_rules": line.flight_rules,
                        "start_time": line.start_time.dt.strftime('%d-%H:%M'),
                        "end_time": line.end_time.dt.strftime('%d-%H:%M'),
                    }
                    for line in taf.data.forecast
                ],
            }
        else:
            data["taf"] = None
        
        airport_data[airport_code] = data
    return airport_data

async def main():
    airports = large_airports_df['gps_code'].tolist()
    data = await fetch_airport_data_async(airports)
    # html_output = generate_summary_html_safe(data)
#     # with open("weather_summary.html", "w") as file:
#     #     file.write(html_output)
    return data  # Return the data instead of printing it

# Run the Main Function
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    data = loop.run_until_complete(main())  # Store the result in a variable



In [26]:
data

{'KABQ': {'metar': {'raw': 'KABQ 290152Z 10012KT 10SM FEW140 FEW250 10/M11 A3010 RMK AO2 SLP180 T01001106 $',
   'flight_rules': 'VFR',
   'summary': 'Winds E-100 at 12kt, Vis 10sm, Temp 10°C, Dew -11°C, Alt 30.10 inHg, Few clouds at 14000ft, Few clouds at 25000ft'},
  'taf': {'raw': TafTrans(forecast=[TafLineTrans(altimeter='', clouds='Sky clear', wx_codes='', visibility='Greater than 6sm ( >10km )', icing='', turbulence='', wind='SE-130 at 11kt', wind_shear=''), TafLineTrans(altimeter='', clouds='Scattered clouds at 9000ft - Reported AGL', wx_codes='', visibility='Greater than 6sm ( >10km )', icing='', turbulence='', wind='E-100 at 12kt', wind_shear=''), TafLineTrans(altimeter='', clouds='Broken layer at 3500ft - Reported AGL', wx_codes='Vicinity Showers', visibility='Greater than 6sm ( >10km )', icing='', turbulence='', wind='E-080 at 7kt', wind_shear=''), TafLineTrans(altimeter='', clouds='Scattered clouds at 7000ft - Reported AGL', wx_codes='', visibility='Greater than 6sm ( >10km

In [27]:
# Initialize a list to hold airports with icing conditions
airports_with_icing = []

# Iterate through the dictionary
for airport_code, weather_info in data.items():
    if 'taf' in weather_info and weather_info['taf'] is not None:  # Ensure TAF data is available
        for forecast_line in weather_info['taf']['raw'].forecast:  # Loop through each forecast line
            if forecast_line.icing:  # Check if icing is mentioned
                # Since icing is an empty string in your example, you'd check its presence differently
                # This is a placeholder condition, adjust according to how icing data is represented
                if forecast_line.icing != '':  # Adjust this condition based on actual data representation
                    airports_with_icing.append(airport_code)
                    break  # Stop checking further lines for this airport

# Now, 'airports_with_icing' contains the codes of airports with icing conditions
print("Airports with icing conditions:", airports_with_icing)

Airports with icing conditions: ['KCBM', 'KCVS', 'KHMN', 'KLTS', 'KMUO', 'KTCM', 'KWRB']


## Generates HTML for airports 



In [ ]:
# # Function to Generate HTML
# def generate_summary_html_safe(airports_data):
#     # Styles
#     styles = """
#     <style>
#         table {
#             border-collapse: collapse;
#             width: 80%;
#             margin: 20px auto;
#         }
#         th, td {
#             text-align: left;
#             padding: 8px 12px;
#             border: 1px solid #ddd;
#         }
#         tr:nth-child(even) {
#             background-color: #f4f4f4;
#         }
#         th {
#             background-color: #f2f2f2;
#         }
#         .airport-name {
#             font-weight: bold;
#             font-size: 1.5em;
#             text-align: center;
#             margin-top: 20px;
#         }
#         .section-header {
#             font-weight: bold;
#             margin-top: 40px;
#             font-size: 1.3em;
#             text-align: center;
#         }
#     </style>
#     """
    
#     # Rows
#     metar_rows_html = ""
#     taf_rows_html = ""
#     for airport, data in airports_data.items():
#         # METAR
#         if data["metar"]:
#             metar = data["metar"]["summary"].split(", ")
#             metar_rows_html += f"<tr>"
#             metar_rows_html += f"<td>{airport}</td>"
#             metar_rows_html += f"<td>{data['metar']['flight_rules']}</td>"
#             for item in metar:
#                 metar_rows_html += f"<td>{item}</td>"
#             metar_rows_html += "</tr>"
        
#         # TAF
#         if data["taf"]:
#             taf_rows_html += f"<tr>"
#             taf_rows_html += f"<td>{airport}</td>"
#             taf_forecast = ', '.join([f"{forecast['flight_rules']} from {forecast['start_time']} to {forecast['end_time']}" for forecast in data['taf']['forecast']])
#             taf_rows_html += f"<td>{taf_forecast}</td>"
#             taf_rows_html += "</tr>"

#     # Complete HTML Structure
#     html = f"""
#     <html>
#         <head>
#             <title>Weather Summary</title>
#             {styles}
#         </head>
#         <body>
#             <div class='airport-name'>Weather Summary for {', '.join(airports_data.keys())}</div>
#             <div class='section-header'>METAR</div>
#             <table>
#                 <tr><th>Airport</th><th>Flight Rules</th><th>Wind</th><th>Visibility</th><th>Temperature</th><th>Dew Point</th><th>Pressure</th><th>Clouds</th><th>Conditions</th></tr>
#                 {metar_rows_html}
#             </table>
#             <div class='section-header'>TAF</div>
#             <table>
#                 <tr><th>Airport</th><th>Forecast</th></tr>
#                 {taf_rows_html}
#             </table>
#         </body>
#     </html>
#     """
#     return html

# Async Main Function to Drive the Process
# Async Main Function to Drive the Process

In [ ]:
import nest_asyncio
nest_asyncio.apply()

taf = avwx.Taf('KJFK')

# Use asyncio.run() to run taf.update()
asyncio.run(taf.update())
print(taf.raw)




In [ ]:



taf = avwx.Taf('KJFK')

# Use asyncio.run() to run taf.update()
asyncio.run(taf.update())
print(taf.raw)

# Models

# Use the avwx library models for for forecasting